 # Phase 1 - 기본 전처리  
 데이터셋 구성 및 결측치 확인
 

In [ ]:
from google.colab import files

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
files.upload();

In [ ]:
df = pd.read_csv('u.data', sep = '\t', header = None)

In [ ]:
df.head

<bound method NDFrame.head of          0     1  2          3
0      196   242  3  881250949
1      186   302  3  891717742
2       22   377  1  878887116
3      244    51  2  880606923
4      166   346  1  886397596
...    ...   ... ..        ...
99995  880   476  3  880175444
99996  716   204  5  879795543
99997  276  1090  1  874795795
99998   13   225  2  882399156
99999   12   203  3  879959583

[100000 rows x 4 columns]>

In [ ]:
# 데이터셋 칼럼 지정
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']

In [ ]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
df['timestamp']

0        881250949
1        891717742
2        878887116
3        880606923
4        886397596
           ...    
99995    880175444
99996    879795543
99997    874795795
99998    882399156
99999    879959583
Name: timestamp, Length: 100000, dtype: int64

In [ ]:
# timestamp 날짜로 변환
from datetime import datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [ ]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [ ]:
# 5점 rating 이상 없음
df.rating.unique()

array([3, 1, 2, 4, 5])

In [ ]:
# 결측치 없음(user별로 모든 영화에 대한 평점이 존재하는 건 아님)
df.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
import chardet

with open('u.item', 'rb') as f:
  enc = chardet.detect(f.read())

In [ ]:
enc

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [ ]:
# chardet 안쓰면 인코더(utf-8)로 안 읽힘 - chardet: 읽어보고 가능한 인코딩 알아주는 기능
df_item = pd.read_csv('u.item', header = None, encoding = enc['encoding'], sep = '|')
df_item.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
item_columns = ['movie_id', 'movie_title', 'release_date',  'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", "Comedy", "Crime", "Documentary",
                "Drama", 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [ ]:
df_item.columns = item_columns

In [ ]:
df_item[df_item['release_date'].isnull()]

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
266,267,unknown,NaN,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_item = df_item.drop(columns = ['video_release_date', 'IMDb_URL'])

In [ ]:
# 이름과 날짜 없는 행 1개 제거
df_item = df_item.drop(axis = 0, index =266)

In [ ]:
# 영화 이름 뒤 연도 제거
df_item['movie_title'] = df_item['movie_title'].str[:-7]

In [ ]:
df_item['release_date'].str[3:6].unique()

array(['Jan', 'Aug', 'Oct', 'Feb', 'Mar', 'May', 'Jun', 'Apr', 'Jul',
       'Sep', 'Nov', 'Dec', 'eb-'], dtype=object)

In [ ]:
df_item.loc[1372,'release_date'] = '04-Feb-1971'

In [ ]:
df_item.loc[1372,'release_date']

'04-Feb-1971'

In [ ]:
df_item['release_date'].str[3:6].unique()

array(['Jan', 'Aug', 'Oct', 'Feb', 'Mar', 'May', 'Jun', 'Apr', 'Jul',
       'Sep', 'Nov', 'Dec'], dtype=object)

In [ ]:
df_item['release_date'].str[:2].unique()

In [ ]:
df_item['release_date'].str[-4:].unique()

In [ ]:
month = {
    'Jan' : '01',
    'Feb' : '02',
    'Mar' : '03',
    'Apr' : '04',
    'May' : '05',
    'Jun' : '06',
    'Jul' : '07',
    'Aug' : '08',
    'Sep' : '09',
    'Oct' : '10',
    'Nov' : '11',
    'Dec' : '12'
}

In [ ]:
# release date 숫자 표기로 변환
month_numed = pd.Series([month[x] for x in df_item['release_date'].str[3: 6]])

In [ ]:
len(df_item)

1681

In [ ]:
len(month_numed)

1681

In [ ]:
df_item['release_date'].tail()

1676    06-Feb-1998
1677    06-Feb-1998
1678    01-Jan-1998
1679    01-Jan-1994
1680    08-Mar-1996
Name: release_date, dtype: object

In [ ]:
df_item.reset_index(inplace=True)

In [ ]:
month_numed.tail()

1676    02
1677    02
1678    01
1679    01
1680    03
dtype: object

In [ ]:
month_numed.unique()

array(['01', '08', '10', '02', '03', '05', '06', '04', '07', '09', '11',
       '12'], dtype=object)

In [ ]:
df_item['release_date'] = df_item['release_date'].str[-4:]+month_numed+df_item['release_date'].str[:2]

In [ ]:
df_item['release_date'] = pd.to_datetime(df_item['release_date'], format = '%Y%m%d')

In [ ]:
df_item.tail()

,index,movie_id,movie_title,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1676,1677,1678,Mat' i syn,1998-02-06,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1677,1678,1679,B. Monkey,1998-02-06,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1678,1679,1680,Sliding Doors,1998-01-01,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1679,1680,1681,You So Crazy,1994-01-01,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1680,1681,1682,Scream of Stone (Schrei aus Stein),1996-03-08,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.describe()

In [ ]:
df_item[['unknown', 'Action',
       'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].describe()

In [ ]:
df_user = pd.read_csv('u.user', header = None, sep = "|")

In [ ]:
df_user.head()

,0,1,2,3,4
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
usercol = ['user_id','age','gender','occupation','zip_code']
df_user.columns = usercol
df_user.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
df_user.isnull().sum()

user_id       0
age           0
gender        0
occupation    0
zip_code      0
dtype: int64

In [ ]:
df_user.duplicated().sum()

0

In [ ]:
len(df.user_id.unique())

943

In [ ]:
len(df_user.user_id.unique())

943

In [ ]:
df_user.gender.unique()

array(['M', 'F'], dtype=object)

In [ ]:
#df_user는 결측치, 이상값 없음
df_user.occupation.unique()

array(['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor'],
      dtype=object)

In [ ]:
df.to_csv('ratings.csv', index = False)

In [ ]:
test = pd.read_csv('users.csv')
test.head()

In [ ]:
df_item.to_csv('movies.csv', index = False)

In [ ]:
df_item = df_item.drop(columns = ['index'])

In [ ]:
df_user.to_csv('users.csv', index = False)